# Products Clusters

In [5]:
# Import packages
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Autoreload
%load_ext autoreload
%autoreload 2

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Import data
from bimbo.data import Bimbo

data = Bimbo().get_data()

# Load in the train and test datasets
# train_df = data['train']
# test_df = data['test']
# cliente_tabla_df = data['cliente_tabla']
producto_tabla_df = data['producto_tabla']
# town_state_df = data['town_state']
# sample_submission_df = data['sample_submission']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Memory usage of dataframe is 0.04 MB
Memory usage after optimization is: 0.11 MB
Decreased by -184.7%
Memory usage of dataframe is 14.27 MB
Memory usage after optimization is: 19.51 MB
Decreased by -36.7%
Memory usage of dataframe is 373.80 MB
Memory usage after optimization is: 120.15 MB
Decreased by 67.9%
Memory usage of dataframe is 0.02 MB
Memory usage after optimization is: 0.02 MB
Decreased by 6.0%
Memory usage of dataframe is 6225.47 MB
Memory usage after optimization is: 2122.32 MB
Decreased by 65.9%
Memory usage of dataframe is 106.80 MB
Memory usage after optimization is: 33.38 MB
Decreased by 68.7%


In [6]:
producto_tabla_df.head(3)

,Producto_ID,NombreProducto
0,0,NO IDENTIFICADO 0
1,9,Capuccino Moka 750g NES 9
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41


### Text pre-processing

In [9]:
from bimbo.text_preprocess import Preprocess

In [10]:
text = "Capuccino Moka 750g NES 9"

In [11]:
pr = Preprocess()

In [16]:
lower_text = pr.to_lower(text)
sentence_tokens = pr.sentence_tokenize(lower_text)
for each_sent in sentence_tokens:
    lemmatizzed_sent = pr.lemmatize(each_sent)
    clean_text = pr.remove_numbers(lemmatizzed_sent)
    clean_text = pr.remove_punct(clean_text)
    clean_text = pr.remove_Tags(clean_text)
    clean_text = pr.remove_stopwords(clean_text)
    word_tokens = pr.word_tokenize(clean_text)
    print(word_tokens)

['capuccino', 'moka', 'g', 'ne']


In [14]:
print(producto_tabla_df['NombreProducto'])

0                                    NO IDENTIFICADO 0
1                            Capuccino Moka 750g NES 9
2               Bimbollos Ext sAjonjoli 6p 480g BIM 41
3                       Burritos Sincro 170g CU LON 53
4                  Div Tira Mini Doradita 4p 45g TR 72
5                    Pan Multigrano Linaza 540g BIM 73
6                         Tostado Integral 180g WON 98
7                               Pan Blanco 567g WON 99
8               Super Pan Bco Ajonjoli 680g SP WON 100
9                   Wonder 100pct mediano 475g WON 106
10                   Wonder 100pct gde 680g SP WON 107
11                 Baguette Precocida Cong 280g DH 108
12                        Pan Multicereal 475g WON 109
13                       Tostado Integral 180g WON 112
14            Biscotel Receta Original 410g CU SUA 122
15                    Super Bollos 5in 8p 540g WON 123
16                              Bollos 8p 450g WON 125
17                      Bollos BK 4in 36p 1635g SL 131
18        

In [15]:
# Found on Kaggle (https://www.kaggle.com/vykhand/exploring-products)
# Credit: Andrey Vykhodtsev

producto_tabla_df['short_name'] = producto_tabla_df.NombreProducto.str.extract('^(\D*)', expand=False)
producto_tabla_df['brand'] = producto_tabla_df.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
w = producto_tabla_df.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
producto_tabla_df['weight'] = w[0].astype('float')*w[1].map({'Kg':1000, 'g':1})
producto_tabla_df['pieces'] =  producto_tabla_df.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')
producto_tabla_df.head()

,Producto_ID,NombreProducto,short_name,brand,weight,pieces
0,0,NO IDENTIFICADO 0,NO IDENTIFICADO,IDENTIFICADO,NaN,NaN
1,9,Capuccino Moka 750g NES 9,Capuccino Moka,NES,750.0,NaN
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41,Bimbollos Ext sAjonjoli,BIM,480.0,6.0
3,53,Burritos Sincro 170g CU LON 53,Burritos Sincro,LON,170.0,NaN
4,72,Div Tira Mini Doradita 4p 45g TR 72,Div Tira Mini Doradita,TR,45.0,4.0


The **short_name** and the **brand** are important.

In [17]:
producto_tabla_df["ProcessedText"] = producto_tabla_df.short_name.apply(lambda x : pr.preprocess(x))
producto_tabla_df["ProcessedText"]

0                       [identificado]
1                    [capuccino, moka]
2          [bimbollos, ext, sajonjoli]
3                    [burrito, sincro]
4          [div, tira, mini, doradita]
5            [pan, multigrano, linaza]
6                  [tostado, integral]
7                        [pan, blanco]
8          [super, pan, bco, ajonjoli]
9                             [wonder]
10                            [wonder]
11         [baguette, precocida, cong]
12                  [pan, multicereal]
13                 [tostado, integral]
14        [biscotel, receta, original]
15                     [super, bollos]
16                            [bollos]
17                        [bollos, bk]
18                        [bollos, bk]
19                        [bollos, bk]
20                        [bollos, bk]
21                          [hot, dog]
22                   [super, hot, dog]
23                    [medias, noches]
24                         [bollo, wb]
25                     [m

In [18]:
# # SHOULD I ADD MORE STOP WORDS ???? (and take them away ????)
# stop = ['g','p','won','mla','bim','lon','cc', 'ml', 'dh', 'sl', 'ne', 'bk', 'in', 'tir', 'mta', 'tr', 'kg']

# producto_tabla_df["ProcessedText"] = producto_tabla_df["ProcessedText"]\
#                                     .apply(lambda x: [item for item in x if item not in stop])
# producto_tabla_df["ProcessedText"]

### product clustering

In [19]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

vocab_size = 10000 # maximum number of words to keep (keeps the most frequent)
maxlen = 200 # maximum number of words in a review

Using TensorFlow backend.


In [20]:
tokenizer = Tokenizer(num_words=maxlen)
tokenizer.fit_on_texts(list(producto_tabla_df["ProcessedText"].values))
X_train = tokenizer.texts_to_sequences(producto_tabla_df["ProcessedText"] .values)
x_train = sequence.pad_sequences(X_train, maxlen = maxlen)

In [21]:
x_train

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,  31],
       ...,
       [  0,   0,   0, ...,   0,  25, 145],
       [  0,   0,   0, ...,   0,  25, 145],
       [  0,   0,   0, ...,   0,  25, 145]], dtype=int32)

In [22]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=42)
kmeans.fit(x_train)
y_kmeans = kmeans.predict(x_train)

In [23]:
y_kmeans

array([ 0,  0, 11, ...,  2,  2,  2], dtype=int32)

In [24]:
np.unique(y_kmeans)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41], dtype=int32)

In [25]:
cluster_df = pd.DataFrame(y_kmeans, columns=['cluster'])
cluster_df.head(3)

,cluster
0,0
1,0
2,11


In [26]:
producto_clusters_df = pd.concat([producto_tabla_df, cluster_df + 1], axis=1)
producto_clusters_df

,Producto_ID,NombreProducto,short_name,brand,weight,pieces,ProcessedText,cluster
0,0,NO IDENTIFICADO 0,NO IDENTIFICADO,IDENTIFICADO,NaN,NaN,[identificado],1
1,9,Capuccino Moka 750g NES 9,Capuccino Moka,NES,750.0,NaN,"[capuccino, moka]",1
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41,Bimbollos Ext sAjonjoli,BIM,480.0,6.0,"[bimbollos, ext, sajonjoli]",12
3,53,Burritos Sincro 170g CU LON 53,Burritos Sincro,LON,170.0,NaN,"[burrito, sincro]",25
4,72,Div Tira Mini Doradita 4p 45g TR 72,Div Tira Mini Doradita,TR,45.0,4.0,"[div, tira, mini, doradita]",7
5,73,Pan Multigrano Linaza 540g BIM 73,Pan Multigrano Linaza,BIM,540.0,NaN,"[pan, multigrano, linaza]",9
6,98,Tostado Integral 180g WON 98,Tostado Integral,WON,180.0,NaN,"[tostado, integral]",4
7,99,Pan Blanco 567g WON 99,Pan Blanco,WON,567.0,NaN,"[pan, blanco]",1
8,100,Super Pan Bco Ajonjoli 680g SP WON 100,Super Pan Bco Ajonjoli,WON,680.0,NaN,"[super, pan, bco, ajonjoli]",35
9,106,Wonder 100pct mediano 475g WON 106,Wonder,WON,475.0,NaN,[wonder],9


### Write CSV

In [18]:
producto_clusters_df.to_csv('../data/producto_clusters.csv',index=False)